In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import nltk
nltk.download('stopwords')
import re
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
source_path = '/content/drive/My Drive/Colab Notebooks/balanced_reviews.csv'
destination_path = '/content/balanced_reviews.csv'
shutil.copy(source_path, destination_path)

'/content/balanced_reviews.csv'

In [ ]:
df = pd.read_csv('/content/balanced_reviews.csv') # Remove index_col='id'
df = df.set_index('id') if 'id' in df.columns else df #Set 'id' as index if it exists.
print(df.head())

                                              Review  Label
0  I mostly drink English Tea and it is very yumm...      1
1  Bought this for a 6 yr old with a sore mouth (...      1
2  I highly recommend this product, I hope they g...      1
3  I just want to say as well as express my true ...      1
4  Click is a wonderful source of protein. Not th...      1


In [ ]:
def datapreprocess(sen):

    sen = re.sub(r"didn't", "did not", sen)
    sen = re.sub(r"don't", "do not", sen)
    sen = re.sub(r"won't", "will not", sen)
    sen = re.sub(r"can't", "can not", sen)
    sen = re.sub(r"wasn't", "was not", sen)
    sen = re.sub(r"\'ve", " have", sen)
    sen = re.sub(r"\'m", " am", sen)
    sen = re.sub(r"\'ll", " will", sen)
    sen = re.sub(r"\'re", " are", sen)
    sen = re.sub(r"\'s", " is", sen)
    sen = re.sub(r"\'d", " would", sen)
    sen = re.sub(r"\'t", " not", sen)
    sen = re.sub(r"\'m", " am", sen)


    p = set(string.punctuation) # p take all the punctuations, punctuations displayed below
    sen=sen.lower() # convert text to lower case
    words=sen.split() # split the text into words
    ctext=[]
    for i in range(10): #adding numbers from 0 - 9 to p
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        ctext.append(t)
    return " ".join([i for i in ctext]) # joining the cleaned words to text



stop=set(stopwords.words('english'))
print(stop)
stop.remove('no')
stop.remove('not')
stop.remove('nor')

preprocessed_essays = []
# Use 'df' instead of 'data' to refer to the DataFrame
# The column name is 'Review', not 'balanced_reviews'
for sentance in df['Review'].values:  # Changed 'balanced_reviews' to 'Review'
    try:
        ctxt=datapreprocess(str(sentance))
        if len(ctxt)==0: raise()
        ctxt = ' '.join(e for e in ctxt.split() if e not in stop)
        preprocessed_essays.append(ctxt.lower().strip())
    except:
        preprocessed_essays.append("NAN")
df['cleanedtext'] = preprocessed_essays # Changed 'data' to 'df'
df.drop(df[df["cleanedtext"]=="NAN"].index,axis = 0,inplace = True) # deleting rows that have no text, Changed 'data' to 'df'
df=df.reset_index(drop='true') # after deleting rows index should be reset, Changed 'data' to 'df'
# Assuming 'Label' column contains 'positive' and 'negative' values
df['decision']= df['Label'].replace('positive',1) # Changed 'decision' to 'Label'
df['decision']= df['Label'].replace('negative',0) # Changed 'decision' to 'Label'


s = SnowballStemmer("english")
p=[]
for i in df['cleanedtext'].values: # Changed 'data' to 'df'
    k=[]
    for word in i.split():
        k.append(s.stem(word))
    l=' '.join(k)
    p.append(l)

df['finaltext']=p # Changed 'data' to 'df'

df.to_csv (r'balanced_reviews.csv')

{'ll', 've', 's', 'to', 'all', "needn't", 'by', 'ours', 'both', 'under', 'but', 'has', "she'll", 'what', "he'd", 'this', 'too', "i'll", 'than', 'own', 'having', 'ma', "don't", 'into', 'a', 'wouldn', 'were', 'herself', "isn't", 'are', 'because', 'isn', 'who', 'same', "they've", "we'll", "we'd", 'whom', 'on', "it's", 'above', 'further', 'can', "shan't", 'or', 'off', 'such', 'few', 'their', 'do', 'no', 'it', 'itself', 'an', 'myself', 'where', 'below', "it'll", "hasn't", "you're", 'ourselves', 'doing', "should've", 'have', 'mightn', 'for', 'had', 'if', 'over', 'me', 'so', 'while', 'these', "haven't", 'nor', 'through', 'each', 'those', 'is', 'again', "aren't", 'at', "he'll", 'am', 'couldn', "mightn't", 'doesn', 'other', 'theirs', "we're", 'now', 'before', 'then', 'hadn', 'was', 'you', 'with', "wouldn't", 'her', "they'll", 'down', 'himself', 'i', "she'd", "won't", 'yourself', 'and', 'been', 't', "hadn't", 'm', 'them', 'any', 'don', 'only', 'being', 'yourselves', "weren't", 'once', 'she', "th

In [ ]:
data=pd.read_csv('balanced_reviews.csv')

label = data['decision'].values
feedback = data.drop(['decision'], axis=1)


from sklearn.model_selection import train_test_split
inputtrain, inputtest, outputtrain, outputtest = train_test_split(feedback, label, test_size=0.20, stratify=label)
inputtrain, inputcv, outputtrain, outputcv = train_test_split(inputtrain, outputtrain, test_size=0.20, stratify=outputtrain)
print(inputtrain.shape)
print(inputcv.shape)
print(inputtest.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)

(400060, 5)
(100015, 5)
(125019, 5)
(400060,)
(100015,)
(125019,)


In [ ]:
data=pd.read_csv('balanced_reviews.csv')

label = data['Label'].values
feedback = data.drop(['Label'], axis=1)


from sklearn.model_selection import train_test_split
inputtrain, inputtest, outputtrain, outputtest = train_test_split(feedback, label, test_size=0.20, stratify=label)
inputtrain, inputcv, outputtrain, outputcv = train_test_split(inputtrain, outputtrain, test_size=0.20, stratify=outputtrain)
print(inputtrain.shape)
print(inputcv.shape)
print(inputtest.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)

(400060, 5)
(100015, 5)
(125019, 5)
(400060,)
(100015,)
(125019,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Replace np.nan values with empty strings in 'finaltext' column
inputtrain['finaltext'] = inputtrain['finaltext'].fillna('')
inputcv['finaltext'] = inputcv['finaltext'].fillna('')
inputtest['finaltext'] = inputtest['finaltext'].fillna('')

vec = CountVectorizer(min_df=10,ngram_range=(1,4))
vec.fit(inputtrain['finaltext'].values) # fit has to happen only on train data

train_cleanedtext = vec.transform(inputtrain['finaltext'].values)
cv_cleanedtext = vec.transform(inputcv['finaltext'].values)
test_cleanedtext= vec.transform(inputtest['finaltext'].values)

print("After vectorizations")
print(train_cleanedtext.shape)
print(cv_cleanedtext.shape)
print(test_cleanedtext.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)
print("...............................")

After vectorizations
(400060, 370701)
(100015, 370701)
(125019, 370701)
(400060,)
(100015,)
(125019,)
...............................


In [ ]:
from google.colab import files
files.download('balanced_reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>